# Data Transformation and Ingestion

### Loading Libraries
##### Spark Session, Dataframe Functions, Data types and Json

In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
import json

### Variables Initialization

In [70]:
cassandra_host = "cassandra"
cassandra_user = "cassandra"
cassandra_pwd = "cassandra"
cassandra_port = 9042
cassandra_table = "job"
cassandra_keyspace = "jobs_analytics"
key_space = "jobs_analytics"
table_name = "job"
kafka_server = "kafka:9092"
kafka_topic = "scraped_jobs"
hdfs_folder_path = 'hdfs://namenode:8021/output1/scraped_jobs/'

### Spark Session
##### Spark Session object creation with configuration data stax spark-cassandra connector and cassandra related connectivity credentials.

In [71]:
#Spark Session creation configured to interact with Cassandra
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,com.datastax.spark:spark-cassandra-connector_2.12:3.0.0,com.datastax.spark:spark-cassandra-connector-driver_2.12:3.0.0").\
config("spark.cassandra.connection.host",cassandra_host).\
config("spark.cassandra.auth.username",cassandra_user).\
config("spark.cassandra.auth.password",cassandra_pwd).\
getOrCreate()

### Get data from Kafka with Schema
##### Read data from Kafka topic using Spark structured streaming API by providing Kafka server and Topic details.

In [72]:
#Read data from Kafka topic
job_data = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers",kafka_server)\
  .option("subscribe", kafka_topic)\
  .option("startingOffsets", "earliest")\
  .load()

##### Set the json schema

In [73]:
# Define the schema for your JSON data
json_schema = StructType([
    StructField("TITLE", StringType(), True),  
    StructField("COMPANY_NAME", StringType(), True),
    StructField("JOB_DATE", StringType(), True),
    StructField("JOB_LINK", StringType(), True),
    StructField("JOB_LOCATION", StringType(), True),
    StructField("JOB_SENIORITY_LEVEL", StringType(), True),
    StructField("JOB_EMPLOYMENT_TYPE", StringType(), True),
    StructField("JOB_FUNCTION", StringType(), True),
    StructField("JOB_INDUSTRIES", StringType(), True),
    StructField("NUMBER_APPLICANTS", StringType(), True),
    StructField("JOB_DESCRIPTION", StringType(), True),
    StructField("NUMBER_JOBS", StringType(), True)
])

##### Convert the key and the value of the Kafka message (the actual sent data, typically serialized as JSON) to a string. It also assigns an alias "json_data" to the column value.
##### Selec the data using the json_schema we set

In [74]:
def is_integer(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

job_data = job_data.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING) as json_data")\
    .select(from_json("json_data", json_schema).alias("data"))\
    .select("data.TITLE", "data.COMPANY_NAME", "data.JOB_DATE", "data.JOB_LINK", "data.JOB_LOCATION", 
        "data.JOB_SENIORITY_LEVEL", "data.JOB_EMPLOYMENT_TYPE", "data.JOB_FUNCTION", "data.JOB_INDUSTRIES", 
        "data.NUMBER_APPLICANTS", "data.JOB_DESCRIPTION")  # didn't select  "data.NUMBER_JOBS"

# convert JOB_DATE into date column
job_data = job_data.withColumn("JOB_DATE", to_date(job_data["JOB_DATE"], "yyyy-MM-dd"))
    
# Set non-numeric values to null
job_data = job_data.withColumn(
    "NUMBER_APPLICANTS",
    when(col("NUMBER_APPLICANTS").isNotNull() & col("NUMBER_APPLICANTS").rlike(r"^\d+$"), col("NUMBER_APPLICANTS").cast("int")).otherwise(None)
)

In [75]:
job_data

DataFrame[TITLE: string, COMPANY_NAME: string, JOB_DATE: date, JOB_LINK: string, JOB_LOCATION: string, JOB_SENIORITY_LEVEL: string, JOB_EMPLOYMENT_TYPE: string, JOB_FUNCTION: string, JOB_INDUSTRIES: string, NUMBER_APPLICANTS: int, JOB_DESCRIPTION: string]

### Foreach Batch method
##### This method be called from Spark froeachBatch sink and writes to cassandra database. It takes micro batch(dataframe) and its the unique id as input.

In [76]:
def process_row(df, epoch_id):
    """Writes data to Cassandra and HDFS location

    Parameters
    ----------
    df : DataFrame
        Streaming Dataframe
    epoch_id : int
        Unique id for each micro batch/epoch
    """
    df.show(truncate=False)
    df.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table=cassandra_table, keyspace=cassandra_keyspace)\
    .save() 
    df.write.csv(hdfs_folder_path,mode="append") 

### Cassandra Sink
##### Writes stream of delta data to Cassandra using foreachBatch sink continuosly until an interruption occurs. Stores processed indices at a checkpoint location so that it will not process the messages already processed.

In [79]:
#Writes streaming dataframe to ForeachBatch console which ingests data to Cassandra
job_data \
    .writeStream \
    .option("checkpointLocation", "checkpoint/data") \
    .foreachBatch(process_row) \
    .start() \
    .awaitTermination()

+--------------------------------+---------------+----------+----------------------------------------------------------------------------------------------------------------------------+-------------+-------------------+-------------------+--------------------------------------+-------------------------------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

23/10/02 03:30:26 ERROR MicroBatchExecution: Query [id = 3035b345-3517-4ea7-b0d1-359829c6634a, runId = 27415d7d-7da8-44fc-8ac5-3f3e0a4e858f] terminated with error
py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 2442, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/utils.py", line 210, in call
    raise e
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/utils.py", line 207, in call
    self.func(DataFrame(jdf, self.sql_ctx), batch_id)
  File "/tmp/ipykernel_14/1590744430.py", line 15, in process_row
    .options(table=cassandra_table, keyspace=cassandra_keyspace)\
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/readwriter.py", line 825, in save
    self._jwrite.save()
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", l

StreamingQueryException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 2442, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/utils.py", line 210, in call
    raise e
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/utils.py", line 207, in call
    self.func(DataFrame(jdf, self.sql_ctx), batch_id)
  File "/tmp/ipykernel_14/1590744430.py", line 15, in process_row
    .options(table=cassandra_table, keyspace=cassandra_keyspace)\
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/readwriter.py", line 825, in save
    self._jwrite.save()
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/utils.py", line 1